Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots, ApproxFun

In [2]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

-1.6653345369377348e-16 - 0.26835469053190525im
-8.630249292984615e-17 + 0.2683546905319051im
0.03747531973228935 + 0.9999999999999997im
-0.037475319732288806 + 1.0im

Floquet Multipliers = 

0.6651802570066604 - 0.7466828146465888im
0.665180257006661 + 0.7466828146465889im
-1.1249427991479373 + 1.1369156774202889e-15im
-0.8889340869219566 + 1.0886302842019492e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -1 
 
 
 0 
 
 
 1 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,160.843 596.058,160.829 596.049,160.801 596.034,160.759 596.015,160.703 595.991,160.633 595.963,160.55 595.929,160.452 595.891,160.341 595.848,160.217 
 595.8,160.079 595.748,159.928 595.69,159.763 595.628,159.586 595.561,159.396 595.49,159.194 595.413,158.979 595.332,158.752 595.246,158.513 595.155,158.263 
 595.06,158.001 594.959,157.729 594.854,157.446 594.745,157.154 594.63,156.851 594.511,156.539 594.387,156.219 594.258,155.89 594.124,155.554 593.986,155.21 
 593.843,154.86 593.695,154.505 593.543,154.144 593.385,153.779 593.223,153.41 593.057,153.039 592.885,152.665 592.709,152.291 592.528,151.916 592.342,151.543 
 592.152,151.171 591.957,150.803 591.757,150.439 591.553,150.08 591.343,149.728 591.129,149.385 590.911,149.05 590.688,148.726 590.46,148.415 590.227,148.117 
 589.99,147.835 589.747,147.569 589.501,147.322 589.249,147.096 588.993,146.891 588.733,146.71 588.467,146.555 588.197,146.427 587.922,146.329 587.643,146.262 
 587.359,146.228 587.071,146.23 586.777,146.269 586.479,146.348 586.177,146.468 585.87,146.632 585.558,146.841 585.242,147.098 584.921,147.405 584.595,147.764 
 584.265,148.177 583.931,148.646 583.591,149.172 583.248,149.759 582.899,150.407 582.546,151.118 582.189,151.896 581.827,152.74 581.46,153.652 581.089,154.635 
 580.713,155.69 580.333,156.817 579.948,158.018 579.559,159.294 579.165,160.647 578.767,162.076 578.364,163.582 577.957,165.166 577.545,166.828 577.129,168.567 
 576.709,170.385 576.284,172.28 575.854,174.251 575.42,176.299 574.982,178.421 574.539,180.616 574.091,182.884 573.64,185.221 573.184,187.626 572.723,190.096 
 572.258,192.628 571.789,195.22 571.315,197.868 570.837,200.567 570.355,203.315 569.868,206.107 569.377,208.938 568.882,211.803 568.382,214.697 567.878,217.613 
 567.37,220.546 566.857,223.49 566.34,226.437 565.819,229.381 565.293,232.313 564.763,235.227 564.229,238.115 563.691,240.967 563.149,243.775 562.602,246.531 
 562.051,249.224 561.496,251.846 560.936,254.387 560.372,256.836 559.805,259.184 559.233,261.421 558.657,263.536 558.076,265.519 557.492,267.359 556.903,269.046 
 556.31,270.57 555.714,271.919 555.113,273.085 554.508,274.057 553.898,274.826 553.285,275.381 552.668,275.716 552.047,275.82 551.421,275.686 550.792,275.307 
 550.158,274.676 549.521,273.788 548.879,272.637 548.234,271.219 547.585,269.531 546.931,267.572 546.274,265.34 545.612,262.835 544.947,260.059 544.278,257.014 
 543.605,253.705 542.928,250.137 542.247,246.317 541.562,242.252 540.873,237.954 540.181,233.432 539.484,228.699 538.784,223.769 538.08,218.658 537.372,213.382 
 536.66,207.958 535.945,202.406 535.226,196.746 534.503,190.999 533.776,185.187 533.045,179.332 532.311,173.459 531.573,167.591 530.831,161.752 530.086,155.969 
 529.337,150.264 528.584,144.664 527.828,139.192 527.068,133.874 526.304,128.733 525.537,123.792 524.766,119.073 523.992,114.6 523.214,110.391 522.432,106.466 
 521.647,102.845 520.858,99.543 520.066,96.5763 519.27,93.9585 518.471,91.702 517.668,89.8175 516.862,88.3138 516.052,87.1979 515.239,86.4752 514.422,86.1491 
 513.602,86.2209 512.779,86.6904 511.952,87.555 511.122,88.8105 510.288,90.4505 509.451,92.4669 508.611,94.8494 507.768,97.5858 506.921,100.662 506.071,104.062 
 505.217,107.769 504.36,111.761 503.5,116.019 502.637,120.518 501.771,125.234 500.901,130.14 500.028,135.209 499.152,140.41 498.273,145.715 497.391,151.09 
 496.505,156.503 495.617,161.922 494.725,167.313 493.83,172.641 492.932,177.872 492.031,182.974 491.127,187.913 490.22,192.656 489.31,197.174 488.397,201.435 
 487.481,205.412 486.562,209.08 485.64,212.415 484.715,215.398 483.787,218.01 482.856,220.236 481.923,222.067 480.986,223.495 480.047,224.517 479.105,225.133 
 478.16,225.347 477.212,225.167 476.261,224.607 475.307,2